In [1]:
# Import the GFF3 module.
using GFF3
using DataStructures

function prepareLongestCDStranscript(inputFile,outputFile)
    #####################################################
    # 1.get exon CDS and gene info
    #####################################################
    # feature type info
    geneinfoDict = Dict{String,String}()
    mRNAinfoDict = Dict{String,String}()
    exonDict = Dict{String,Int64}()
    CDSDict = Dict{String,Int64}()

    # Open a GFF3 file.
    reader = open(GFF3.Reader, inputFile)

    # Iterate over records.
    for record in reader
        # save gene_id and gene name
        if GFF3.featuretype(record) == "gene"
            gene_id = GFF3.attributes(record,"gene_id")[1]
            try
                geneinfoDict[gene_id] = GFF3.attributes(record,"Name")[1]
            catch
                geneinfoDict[gene_id] = "non-name"
            end
        end
        # save transcript_id and gene_id
        if GFF3.featuretype(record) == "mRNA"
            transcript_id = GFF3.attributes(record,"transcript_id")[1]
            gene_id = split(GFF3.attributes(record,"Parent")[1],":")[2]
            mRNAinfoDict[transcript_id] = gene_id
        end
        # sum up exon length
        if GFF3.featuretype(record) == "exon"
            transid = split(GFF3.attributes(record,"Parent")[1],":")[2]
            exonLength = abs(GFF3.seqend(record) - GFF3.seqstart(record)) + 1
            if !haskey(exonDict,transid)
                exonDict[transid] = exonLength
            else
                exonDict[transid] += exonLength
            end
        end
        # sum up CDS length
        if GFF3.featuretype(record) == "CDS"
            transid = split(GFF3.attributes(record,"Parent")[1],":")[2]
            cdsLength = abs(GFF3.seqend(record) - GFF3.seqstart(record)) + 1
            # println(exonLength)
            if !haskey(CDSDict,transid)
                CDSDict[transid] = cdsLength
            else
                CDSDict[transid] += cdsLength
            end
        end
    end

    # Finally, close the reader
    close(reader)

    #####################################################
    # 2.combine gene and and trascript info
    #####################################################
    # merge info
    merExonDict = DefaultDict(Dict)
    merCDSDict = DefaultDict(Dict)

    # combine gene info with exon length
    for (tranid,exonlength) in exonDict
        if haskey(mRNAinfoDict,tranid)
            key1 = geneinfoDict[mRNAinfoDict[tranid]]*"|"*mRNAinfoDict[tranid]
            merExonDict[key1][tranid] = exonlength
        end
    end
    # combine gene info with cds length
    for (tranid,cdslength) in CDSDict
        if haskey(mRNAinfoDict,tranid)
            key1 = geneinfoDict[mRNAinfoDict[tranid]]*"|"*mRNAinfoDict[tranid]
            merCDSDict[key1][tranid] = cdslength
        end
    end

    #####################################################
    # 3.filter longsest cds-exon transcript
    #####################################################
    # save final longest CDS transcript info
    finalResDict = Dict{String,Vector{Int64}}()

    # filter longest CDS-exon transcript
    for (ginfo,cdsinfo) in merCDSDict
        maxcds = maximum(values(cdsinfo)) # max cds length
        cdslongestDict = Dict{String,Int64}() # save in dict
        for (key,val) in cdsinfo
            if val == maxcds
                cdslongestDict[key] = val
            end
        end
        # whether have one cds longest transcript
        if length(keys(cdslongestDict)) == 1
            longestTranid = [i for i in keys(cdslongestDict)][1]
            # save
            finalResDict["$ginfo|$longestTranid"] = [merExonDict[ginfo][longestTranid],maxcds]
        # which have equal CDS length
        elseif length(keys(cdslongestDict)) > 1
            tmpexondict = merExonDict[ginfo]
            exonlongestDict = Dict{String,Int64}() # save in dict
            for key in keys(cdslongestDict)
                if haskey(tmpexondict,key)
                    exonlongestDict[key] = tmpexondict[key]
                end
            end
            maxexon = maximum(values(exonlongestDict)) # max exon length
            for (key,val) in exonlongestDict
                if val == maxexon
                    # save
                    finalResDict["$ginfo|$key"] = [merExonDict[ginfo][key],maxcds]
                end
            end
        end
    end

    #####################################################
    # 4.save gene other info like chromsome strand and CDS regions
    #####################################################
    longestidTrnscript = Dict{String,String}()
    for key in keys(finalResDict)
        tid = split(key,"|")[3]
        longestidTrnscript[tid] = key
    end
    # save gene other info
    outcdsDict = Dict{String,Vector{String}}()
    outexonDict = Dict{String,Vector{String}}()
    UTR5Dict = Dict{String,Int64}()

    # Open a GFF3 file.
    reader = open(GFF3.Reader, inputFile)

    # Iterate over records.
    for record in reader
        # filter longest transcript gene info
        if GFF3.featuretype(record) == "CDS"
            transid = split(GFF3.attributes(record,"Parent")[1],":")[2]
            if haskey(longestidTrnscript,transid)
                # tags
                id = longestidTrnscript[transid]
                chrom = GFF3.seqid(record)
                strand = GFF3.strand(record)
                seqstart = GFF3.seqstart(record)
                seqend = GFF3.seqend(record)
                key = "$id|$chrom|$strand"
                region = "$seqstart:$seqend"
                # add multiple cds region
                if !haskey(outcdsDict,key)
                    outcdsDict[key] = [region]
                else
                    push!(outcdsDict[key],region)
                end
            end
        end
        # filter longest transcript gene info
        if GFF3.featuretype(record) == "exon"
            transid = split(GFF3.attributes(record,"Parent")[1],":")[2]
            if haskey(longestidTrnscript,transid)
                # tags
                id = longestidTrnscript[transid]
                chrom = GFF3.seqid(record)
                strand = GFF3.strand(record)
                seqstart = GFF3.seqstart(record)
                seqend = GFF3.seqend(record)
                key = "$id|$chrom|$strand"
                region = "$seqstart:$seqend"
                # add multiple exon region
                if !haskey(outexonDict,key)
                    outexonDict[key] = [region]
                else
                    push!(outexonDict[key],region)
                end
            end
        end
        # get 5utr length
        if GFF3.featuretype(record) == "five_prime_UTR"
            transid = split(GFF3.attributes(record,"Parent")[1],":")[2]
            if haskey(longestidTrnscript,transid)
                utr5Length = abs(GFF3.seqend(record) - GFF3.seqstart(record)) + 1
                if !haskey(UTR5Dict,transid)
                    UTR5Dict[transid] = utr5Length
                else
                    UTR5Dict[transid] += utr5Length
                end
            end
        end
    end

    # Finally, close the reader
    close(reader)

    #####################################################
    # 5.finally output tab-delimit file
    # fullid gene_id transcript_id chromosome strand CDS_regions exon_regions 5UTR_length CDS_length 3UTR_length
    #####################################################
    # OR4F5|ENSG00000186092|ENST00000641515|1|+
    outputfile = open(outputFile,"w")

    # output file
    for(key,val) in outcdsDict
        gene_name,gene_id,transid,chrom,strand = split(key,"|")
        cds = join(val,",")
        exon = join(outexonDict[key],",")
        if haskey(UTR5Dict,transid) # utr5 length
            utr5length = UTR5Dict[transid] 
        else
            utr5length = 0
        end
        cdslength = CDSDict[transid] # cds length
        utr3length = exonDict[transid] - (utr5length + cdslength) # utr3 length
        write(outputfile,join([key,gene_name,gene_id,transid,chrom,strand,cds,exon,utr5length,cdslength,utr3length],"\t")*"\n")
    end
    # close file
    close(outputfile)
end

prepareLongestCDStranscript (generic function with 1 method)

In [2]:
# prepareLongestCDStranscript("Homo_sapiens.GRCh38.106.gff3","longestCDSinfo.txt")
prepareLongestCDStranscript("Homo_sapiens.GRCh38.106.gff3","longestCDSinfo.txt")